### Yielding and Generators

Let's start by writing a "simple" iterator first using the techniques we learned in the previous section.

In [ ]:
import math

In [ ]:
class FactIter:
    def __init__(self, n):
        self.n = n
        self.i = 0

    def __iter__(self):
        return self

    def __next__(self):
        if self.i >= self.n:
            raise StopIteration
        else:
            result = math.factorial(self.i)
            self.i += 1
            return result

In [ ]:
fact_iter = FactIter(5)

In [ ]:
for num in fact_iter:
    print(num)

1
1
2
6
24


We could achieve the same thing using the `iter` method's second form - we just have to know our sentinel value - in this case it would be the factorial of n+1 where n is the last integer's factorial we want our iterator to produce:

In [ ]:
def fact():
    i = 0
    def inner():
        nonlocal i
        result = math.factorial(i)
        i += 1
        return result
    return inner           

In [ ]:
fact_iter = iter(fact(), math.factorial(5))

In [ ]:
for num in fact_iter:
    print(num)

1
1
2
6
24


You'll note that in both cases `fact_iter` was an **iterator**. In the first example we implemented the iterator ourselves, in the second example Python built-it for us.

The second example was a little less code, but maybe a little more difficult to understand if we were just shown the code without having written it ourselves.

There has to be a better way!!

And indeed, there is... generators.

Let's look at the `yield` statement first.

The `yield` statement is used almost like a `return` statement in a function - but there is a huge difference - when the `yield` statement is encountered, Python returns whatever value `yield` specifies, but it "pauses" execution of the function. We can then "call" the same function again and it will "resume" from where the last `yield` was encountered.

I say "call" because we do not "resume" the function by calling it - instead we use the function... `next()` !!!

Let's try it:

In [ ]:
def my_func():
    print('line 1')
    yield 'Flying'
    print('line 2')
    yield 'Circus'    

In [ ]:
my_func()

<generator object my_func at 0x0000019DA77D3BA0>

So, executing `my_func()`, returned a generator object - it did not actually "run" the body of `my_func` (none of our print statements actually ran).

To do that, we need to use the `next()` function. 

`next()`?? Isn't that what we use for iteration??

In [ ]:
gen_my_func = my_func()

In [ ]:
next(gen_my_func)

line 1


'Flying'

In [ ]:
next(gen_my_func)

line 2


'Circus'

And let's call it one more time:

In [ ]:
next(gen_my_func)

StopIteration: 

A `StopIteration` exception.

Hmmm... `next`, `StopIteration`? What does this look like? 

An **iterator**!

And in fact that's exactly what Python generators are - they **are** iterators. 

If generators are iterators, they should implement the iterator **protocol**.

Let's see:

In [ ]:
gen_my_func = my_func()

In [ ]:
'__iter__' in dir(gen_my_func)

True

In [ ]:
'__next__' in dir(gen_my_func)

True

And so we just have an iterator, which we can use with the `iter()` function and the `next()` function like any other iterator:

In [ ]:
gen_my_func

<generator object my_func at 0x0000019DA78660A0>

In [ ]:
iter(gen_my_func)

<generator object my_func at 0x0000019DA78660A0>

As you can see, the `iter` function returned the same object - something we expect with iterators.

So if this is an iterator that Python builds, how does it know when to stop the iteration (raise the `StopIteration` exception)?

In the example above, it seemed clear - when the function finished running - there were no more statements after that last `yield`.

What actually happens if a function finishes running and we don't explicitly return something?

Remember that Python fills in the gap, and returns `None`.

In general, the iteration will terminate when we **return** something from the function.

Let's take a look:

In [ ]:
def squares(sentinel):
    i = 0
    while True:
        if i < sentinel:
            result = i**2
            i += 1
            yield result
        else:
            return 'all done!'

In [ ]:
sq = squares(3)

In [ ]:
next(sq)

0

In [ ]:
next(sq)

1

In [ ]:
next(sq)

4

In [ ]:
next(sq)

StopIteration: all done!

And the return value of our function became the message of the `StopIteration` exception.

But, we can simplify this slightly:

In [ ]:
def squares(sentinel):
    i = 0
    while True:
        if i < sentinel:
            yield i**2
            i += 1 # note how we can incremenet **after** the yield
        else:
            return 'all done!'

In [ ]:
for num in squares(5):
    print(num)

0
1
4
9
16


So now let's see how we could re-write our initial `factorial` example:

In [ ]:
def factorials(n):
    for i in range(n):
        yield math.factorial(i)    

In [ ]:
for num in factorials(5):
    print(num)

1
1
2
6
24


Now that's a much simpler and understandable way to create the iterator!

Note that a generator **is** an iterator, but not vice-versa - iterators are not necessarily generators, just like sequences are iterables, but iterables are not necessarily sequences.

Another thing to note is that since generators are iterators, they also  become exhausted (consumed) just like an iterator does.

In [ ]:
facts = factorials(5)

In [ ]:
list(facts)

[1, 1, 2, 6, 24]

In [ ]:
list(facts)

[]

As you can see, our second iteration through the same generator ended up with nothing - that's because the generator has been exhausted:

In [ ]:
next(facts)

StopIteration: 

### Example: Fibonacci Sequence

Here is the Fibonacci sequence:

```
1 1 2 3 5 8 13 ...
```

As you can see there is a recursive definition of the numbers in this sequence:

```
Fib(n) = Fib(n-1) + Fib(n-2)
```
where 

```
Fib(0) = 1
``` 

and

```
Fib(1) = 1
```

Although we can certainly use a recursive approach to calculate the *n-th* number in the sequence, it is not a very effective method - we can of course help it by using memoization, but we'll still run in Python's maximum recursion depth (which we can change also) - but overall it's not very efficient:

In [ ]:
def fib_recursive(n):
    if n <= 1:
        return 1
    else:
        return fib_recursive(n-1) + fib_recursive(n-2)

In [ ]:
[fib_recursive(i) for i in range(7)]

[1, 1, 2, 3, 5, 8, 13]

But this quickly becomes an issue as `n` grows larger:

In [ ]:
from timeit import timeit

In [ ]:
timeit('fib_recursive(10)', globals=globals(), number=10)

0.00027306209887231856

In [ ]:
timeit('fib_recursive(28)', globals=globals(), number=10)

1.5438638503706388

In [ ]:
timeit('fib_recursive(29)', globals=globals(), number=10)

2.507533317368592

We can alleviate this by using memoization:

In [ ]:
from functools import lru_cache

In [ ]:
@lru_cache()
def fib_recursive(n):
    if n <= 1:
        return 1
    else:
        return fib_recursive(n-1) + fib_recursive(n-2)

In [ ]:
timeit('fib_recursive(10)', globals=globals(), number=10)

9.75221781729374e-06

In [ ]:
timeit('fib_recursive(29)', globals=globals(), number=10)

1.9775330573068572e-05

As you can see, performance is greatly improved, but we still have a recursion depth limit:

In [ ]:
@lru_cache()
def fib_recursive(n):
    if n <= 1:
        return 1
    else:
        return fib_recursive(n-1) + fib_recursive(n-2)

In [ ]:
fib_recursive(2000)

RecursionError: maximum recursion depth exceeded while calling a Python object

So we can use a non-recursive approach to calculate the `n-th` Fibonacci number:

In [ ]:
def fib(n):
    fib_0 = 1
    fib_1 = 1
    for i in range(n-1):
        fib_0, fib_1 = fib_1, fib_0 + fib_1
    return fib_1

In [ ]:
[fib(i) for i in range(7)]

[1, 1, 2, 3, 5, 8, 13]

This works well for large `n` values too:

In [ ]:
timeit('fib(5000)', globals=globals(), number=10)

0.006382826561139865

So now, let's create an iterator approach so we can iterate over the sequence, but without materializing it (i.e. we want to use lazy evaluation, not eager evaluation)

Our first approach is going to be a custom iterator and iterable:

In [ ]:
class Fib:
    def __init__(self, n):
        self.n = n
        
    def __iter__(self):
        return self.FibIter(self.n)
        
    class FibIter:
        def __init__(self, n):
            self.n = n
            self.i = 0
            
        def __iter__(self):
            return self
        
        def __next__(self):
            if self.i >= self.n:
                raise StopIteration
            else:
                result = fib(self.i)
                self.i += 1
                return result

And we can now iterate the usual way:

In [ ]:
fib_iterable = Fib(7)

In [ ]:
for num in fib_iterable:
    print(num)

1
1
2
3
5
8
13


Of course, we can also use the second form of the `iter` function too, but we have to create a closure first:

In [ ]:
def fib_closure():
    i = 0
    def inner():
        nonlocal i
        result = fib(i)
        i += 1
        return result
    return inner

In [ ]:
fib_numbers = fib_closure()
fib_iter = iter(fib_numbers, fib(7))
for num in fib_iter:
    print(num)

1
1
2
3
5
8
13


But there's two things here:

1. The syntax for either implementation is a little convoluted and not very clear
2. More importantly, notice what happens every time the `next` method is called - it has to calculate every Fibonacci number from scratch (using the `fib` function) - that is wasteful...

Instead, we can use a generator function very effectively here.

Here is our original `fib` function:

In [ ]:
def fib(n):
    fib_0 = 1
    fib_1 = 1
    for i in range(n-1):
        fib_0, fib_1 = fib_1, fib_0 + fib_1
    return fib_1    

In [ ]:
[fib(i) for i in range(7)]

[1, 1, 2, 3, 5, 8, 13]

Now let's modity it into a generator function:

In [ ]:
def fib_gen(n):
    fib_0 = 1
    fib_1 = 1
    for i in range(n-1):
        fib_0, fib_1 = fib_1, fib_0 + fib_1
        yield fib_1    

In [ ]:
[num for num in fib_gen(7)]

[2, 3, 5, 8, 13, 21]

We're almost there. We're missing the first two Fibonacci numbers in the sequence - we need to yield those too.

In [ ]:
def fib_gen(n):
    fib_0 = 1
    yield fib_0
    fib_1 = 1
    yield fib_1
    for i in range(n-1):
        fib_0, fib_1 = fib_1, fib_0 + fib_1
        yield fib_1    

In [ ]:
[num for num in fib_gen(7)]

[1, 1, 2, 3, 5, 8, 13, 21]

And finally we're returning one number too many if `n` is meant to indicate the length of the sequence:

In [ ]:
def fib_gen(n):
    fib_0 = 1
    yield fib_0
    fib_1 = 1
    yield fib_1
    for i in range(n-2):
        fib_0, fib_1 = fib_1, fib_0 + fib_1
        yield fib_1    

And now everything works fine:

In [ ]:
[num for num in fib_gen(7)]

[1, 1, 2, 3, 5, 8, 13]

Let's time it as well to compare it with the other methods:

In [ ]:
timeit('[num for num in Fib(5_000)]', globals=globals(), number=1)

1.4024426054891919

In [ ]:
fib_numbers = fib_closure()
sentinel = fib(5_001)

timeit('[num for num in iter(fib_numbers, sentinel)]', globals=globals(),
      number=1)

1.4315486413535154

In [ ]:
timeit('[num for num in fib_gen(5_000)]', globals=globals(), number=1)

0.0013831895603644284

### Making an Iterable from a Generator

As we now know, generators are iterators.

This means that they become exhausted - so sometimes we want to create an iterable instead.

There's no magic here, we simply have to implement a class that implements the iterable protocol:

Let's write a simple generator that generates the squares of integers:

In [ ]:
def squares_gen(n):
    for i in range(n):
        yield i ** 2

Now, we can create a new generator:

In [ ]:
sq = squares_gen(5)

In [ ]:
for num in sq:
    print(num)

0
1
4
9
16


But, `sq` was an iterator - so now it's been exhausted:

In [ ]:
next(sq)

StopIteration: 

To restart the iteration we have to create a new instance of the generator (iterator):

In [ ]:
sq = squares_gen(5)

In [ ]:
[num for num in sq]

[0, 1, 4, 9, 16]

So, let's wrap this in an iterable:

In [ ]:
class Squares:
    def __init__(self, n):
        self.n = n
        
    def __iter__(self):
        return squares_gen(self.n)

In [ ]:
sq = Squares(5)

In [ ]:
[num for num in sq]

[0, 1, 4, 9, 16]

And we can do it again:

In [ ]:
[num for num in sq]

[0, 1, 4, 9, 16]

We can put those pieces of code together if we prefer:

In [ ]:
class Squares:
    def __init__(self, n):
        self.n = n
        
    @staticmethod
    def squares_gen(n):
        for i in range(n):
            yield i ** 2
        
    def __iter__(self):
        return Squares.squares_gen(self.n)

In [ ]:
sq = Squares(5)

In [ ]:
[num for num in sq]

[0, 1, 4, 9, 16]

#### Generators used with other Generators

I want to point out that you can also easily run into various bug when you use generators with other generator functions.

Consider this example:

In [ ]:
def squares(n):
    for i in range(n):
        yield i ** 2

In [ ]:
sq = squares(5)

In [ ]:
enum_sq = enumerate(sq)

Now `enumerate` is lazy, so `sq` had not, at this point, been consumed:

In [ ]:
next(sq)

0

In [ ]:
next(sq)

1

Since we have consumed two elements from `sq`, when we now use `enumerate` it will have two less elements from sq:

In [ ]:
next(enum_sq)

(0, 4)

You'll notice that we don't get the first element of the original `sq` - instead we get the third element (`2 ** 2`).

Moreover, you'll notice that the index returned in the tuple produced by `enumerate` is 0, not 2!

### Generator Expressions

Recall how list comprehensions worked:

In [ ]:
l = [i ** 2 for i in range(5)]

In [ ]:
l

[0, 1, 4, 9, 16]

The expression inside the `[]` brackets is called a comprehension expression.

The `[]` brackets resulted in a list being created.

We can easily create a **generator** by using `()` parentheses instead of the `[]` brackets:

In [ ]:
g = (i ** 2 for i in range(5))

Note that `g` is a generator, and is also lazily evaluated:

In [ ]:
type(g)

generator

In [ ]:
for item in g:
    print(item)

0
1
4
9
16


And now the generator has been exhausted:

In [ ]:
for item in g:
    print(item)

Scoping works the same way with generator expressions as with list comprehensions, i.e. generator expressions are created by Python using a function, and therefore have local scopes and can access enclosing nonlocal and global scopes.

In [ ]:
import dis

Recall for list comprehensions:

In [ ]:
exp = compile('[i**2 for i in range(5)]', filename='<string>', mode='eval')

In [ ]:
dis.dis(exp)

  1           0 LOAD_CONST               0 (<code object <listcomp> at 0x000002181BDEEDB0, file "<string>", line 1>)
              2 LOAD_CONST               1 ('<listcomp>')
              4 MAKE_FUNCTION            0
              6 LOAD_NAME                0 (range)
              8 LOAD_CONST               2 (5)
             10 CALL_FUNCTION            1
             12 GET_ITER
             14 CALL_FUNCTION            1
             16 RETURN_VALUE


In [ ]:
exp = compile('(i ** 2 for i in range(5))', filename='<string>', mode='eval')

In [ ]:
dis.dis(exp)

  1           0 LOAD_CONST               0 (<code object <genexpr> at 0x000002181BE32150, file "<string>", line 1>)
              2 LOAD_CONST               1 ('<genexpr>')
              4 MAKE_FUNCTION            0
              6 LOAD_NAME                0 (range)
              8 LOAD_CONST               2 (5)
             10 CALL_FUNCTION            1
             12 GET_ITER
             14 CALL_FUNCTION            1
             16 RETURN_VALUE


As you can see the internal mechanism for list comprehensions and generator expressions is almost the same - in particular note how a function is created. The main difference is that in one case a list is created (an iterable), while in the other a generator (an iterator) is produced.

We can iterate over the same list comprehension multiple times, since it is an iterable. However, we can only iterate over a comprehension expression once, since it is an iterator.

In [ ]:
l = [i * 2 for i in range(5)]

In [ ]:
type(l)

list

In [ ]:
g = (i ** 2 for i in range(5))

In [ ]:
type(g)

generator

#### Nested Comprehensions

Just as with list comprehensions, we can nest generator expressions too:

Let's use some of the same example we saw with nested list comprehensions.

##### Example 1

A multiplication table:

Using a list comprehension approach first:

In [ ]:
start = 1
stop = 10

mult_list = [ [i * j 
               for j in range(start, stop+1)]
             for i in range(start, stop+1)]

In [ ]:
mult_list

[[1, 2, 3, 4, 5, 6, 7, 8, 9, 10],
 [2, 4, 6, 8, 10, 12, 14, 16, 18, 20],
 [3, 6, 9, 12, 15, 18, 21, 24, 27, 30],
 [4, 8, 12, 16, 20, 24, 28, 32, 36, 40],
 [5, 10, 15, 20, 25, 30, 35, 40, 45, 50],
 [6, 12, 18, 24, 30, 36, 42, 48, 54, 60],
 [7, 14, 21, 28, 35, 42, 49, 56, 63, 70],
 [8, 16, 24, 32, 40, 48, 56, 64, 72, 80],
 [9, 18, 27, 36, 45, 54, 63, 72, 81, 90],
 [10, 20, 30, 40, 50, 60, 70, 80, 90, 100]]

The equivalent generator expression would be:

In [ ]:
start = 1
stop = 10

mult_list = ( (i * j 
               for j in range(start, stop+1))
             for i in range(start, stop+1))

In [ ]:
mult_list

<generator object <genexpr> at 0x000002181BDD9CA8>

We can iterate through mult_list:

In [ ]:
table = list(mult_list)

In [ ]:
table

[<generator object <genexpr>.<genexpr> at 0x000002181BDD9DB0>,
 <generator object <genexpr>.<genexpr> at 0x000002181BDD9E08>,
 <generator object <genexpr>.<genexpr> at 0x000002181BDD9A98>,
 <generator object <genexpr>.<genexpr> at 0x000002181BDD9D58>,
 <generator object <genexpr>.<genexpr> at 0x000002181BDD96D0>,
 <generator object <genexpr>.<genexpr> at 0x000002181BDD9BF8>,
 <generator object <genexpr>.<genexpr> at 0x000002181BDD9F68>,
 <generator object <genexpr>.<genexpr> at 0x000002181BDD9E60>,
 <generator object <genexpr>.<genexpr> at 0x000002181BDD9BA0>,
 <generator object <genexpr>.<genexpr> at 0x000002181BDD9A40>]

But you'll notice that our rows are themselves generators!

To fully materialize the table we need to iterate through the row generators too:

In [ ]:
table_rows = [list(gen) for gen in table]

In [ ]:
table_rows

[[10, 20, 30, 40, 50, 60, 70, 80, 90, 100],
 [10, 20, 30, 40, 50, 60, 70, 80, 90, 100],
 [10, 20, 30, 40, 50, 60, 70, 80, 90, 100],
 [10, 20, 30, 40, 50, 60, 70, 80, 90, 100],
 [10, 20, 30, 40, 50, 60, 70, 80, 90, 100],
 [10, 20, 30, 40, 50, 60, 70, 80, 90, 100],
 [10, 20, 30, 40, 50, 60, 70, 80, 90, 100],
 [10, 20, 30, 40, 50, 60, 70, 80, 90, 100],
 [10, 20, 30, 40, 50, 60, 70, 80, 90, 100],
 [10, 20, 30, 40, 50, 60, 70, 80, 90, 100]]

Of course, we can mix list comprehensions and generators. 

In this modification, we'll make the rows list comprehensions, and retain the generator expression in the outer comprehension:

In [ ]:
start = 1
stop = 10

mult_list = ( [i * j 
               for j in range(start, stop+1)]
             for i in range(start, stop+1))

Notice what is happening here, the table itself is lazy evaluated, i.e. the rows are not yielded until they are requested - but once a row is requested, the list comprehension that defines the row will be entirely evaluated at that point:

In [ ]:
for item in mult_list:
    print(item)

[1, 2, 3, 4, 5, 6, 7, 8, 9, 10]
[2, 4, 6, 8, 10, 12, 14, 16, 18, 20]
[3, 6, 9, 12, 15, 18, 21, 24, 27, 30]
[4, 8, 12, 16, 20, 24, 28, 32, 36, 40]
[5, 10, 15, 20, 25, 30, 35, 40, 45, 50]
[6, 12, 18, 24, 30, 36, 42, 48, 54, 60]
[7, 14, 21, 28, 35, 42, 49, 56, 63, 70]
[8, 16, 24, 32, 40, 48, 56, 64, 72, 80]
[9, 18, 27, 36, 45, 54, 63, 72, 81, 90]
[10, 20, 30, 40, 50, 60, 70, 80, 90, 100]


##### Example 2

Let's try Pascal's triangle again:

```
1
1 1
1 2 1
1 3 3 1
1 4 6 4 1
```

we just need to know how to calculate combinations:
```
C(n, k) = n! / (k! (n-k)!)
```

* row 0, column 0: n=0, k=0: c(0, 0) = 0! / 0! 0! = 1/1 = 1
* row 4, column 2: n=4, k=2: c(4, 2) = 4! / 2! 2! = 4x3x2 / 2x2 = 6

In other words, we need to calculate the following list of lists:
```
c(0,0)
c(1,0) c(1,1)
c(2,0) c(2,1) c(2,2)
c(3,0) c(3,1) c(3,2) c(3,3)
...
```

Here's how we did it using a list comprehension:

In [ ]:
from math import factorial

def combo(n, k):
    return factorial(n) // (factorial(k) * factorial(n-k))

size = 10  # global variable
pascal = [ [combo(n, k) for k in range(n+1)] for n in range(size+1) ]

In [ ]:
pascal

[[1],
 [1, 1],
 [1, 2, 1],
 [1, 3, 3, 1],
 [1, 4, 6, 4, 1],
 [1, 5, 10, 10, 5, 1],
 [1, 6, 15, 20, 15, 6, 1],
 [1, 7, 21, 35, 35, 21, 7, 1],
 [1, 8, 28, 56, 70, 56, 28, 8, 1],
 [1, 9, 36, 84, 126, 126, 84, 36, 9, 1],
 [1, 10, 45, 120, 210, 252, 210, 120, 45, 10, 1]]

We can now use generator expressions for either one or both of the nested list comprehensions. In this case I'll use it for both:

In [ ]:
size = 10  # global variable
pascal = ( (combo(n, k) for k in range(n+1)) for n in range(size+1) )

If we want to materialize the triangle into a list we'll need to do so ourselves:

In [ ]:
[list(row) for row in pascal]

[[1],
 [1, 1],
 [1, 2, 1],
 [1, 3, 3, 1],
 [1, 4, 6, 4, 1],
 [1, 5, 10, 10, 5, 1],
 [1, 6, 15, 20, 15, 6, 1],
 [1, 7, 21, 35, 35, 21, 7, 1],
 [1, 8, 28, 56, 70, 56, 28, 8, 1],
 [1, 9, 36, 84, 126, 126, 84, 36, 9, 1],
 [1, 10, 45, 120, 210, 252, 210, 120, 45, 10, 1]]

#### Timings

So we see that the main difference between the two approaches is that in one case we have a fully materialized list (i.e. all the elements have been  created and put into list objects), while in the other we are dealing with lazily evaluated iterators.

One main advantage to using generators is that we do not need the up-front calculations - if we end up not consuming the entire iterator, we have saved some time.

The other advantage, as we saw with lazy iteratorsm is that you do not need to have the entire data set in memory at one time. We saw an example of this when reading files - we can read extremely large files one row at a time, without having to store the entire file in memory.

Let's see the time difference between creating a list comprehension and a generator expression for a large Pascal triangle:

In [ ]:
from timeit import timeit

In [ ]:
size = 600

In [ ]:
timeit('[[combo(n, k) for k in range(n+1)] for n in range(size+1)]',
      globals=globals(), number=1)

3.937809023402072

In [ ]:
timeit('((combo(n, k) for k in range(n+1)) for n in range(size+1))',
      globals=globals(), number=1)

3.5216342118005173e-06

As you can see, much faster - but that's because we haven't actually done anything other than set up the nested iterators. Since no iteration took place, no calculations were performed.

In fact, even if we make the inner generator expression a list comprehension, those will not be calculated until the individual rows from the outer generator expression are requested:

In [ ]:
timeit('([combo(n, k) for k in range(n+1)] for n in range(size+1))',
      globals=globals(), number=1)

7.314163362526216e-06

In fact, we can quickly create a **huge** Pascal triangle using the generator approach:

In [ ]:
size = 100_000

timeit('([combo(n, k) for k in range(n+1)] for n in range(size+1))',
      globals=globals(), number=1)

5.959688666123952e-06

What about timing both creating **and** iterating though all the elements?

Let's do this by creating some functions that will do that:

In [ ]:
def pascal_list(size):
    l = [[combo(n, k) for k in range(n+1)] for n in range(size+1)]
    for row in l:
        for item in row:
            pass

In [ ]:
def pascal_gen(size):
    g = ((combo(n, k) for k in range(n+1)) for n in range(size+1))
    for row in g:
        for item in row:
            pass

In [ ]:
size = 600
timeit('pascal_list(size)', globals=globals(), number=1)

3.9256347339324087

In [ ]:
size = 600
timeit('pascal_gen(size)', globals=globals(), number=1)

3.835240885197976

So as you can see, if we actually iterate through each element, we don't end up saving any time - however, creating the iterator is faster, and if we don't use all the elements, then it will be more efficient.

#### Memory Usage

Another thing that is way more efficient is memory usage.

To see this, we'll use a rough technique and the `tracemalloc` standard library module:

In [ ]:
import tracemalloc

In [ ]:
def pascal_list(size):
    l = [[combo(n, k) for k in range(n+1)] for n in range(size+1)]
    for row in l:
        for item in row:
            pass
    stats = tracemalloc.take_snapshot().statistics('lineno')
    print(stats[0].size, 'bytes')

In [ ]:
def pascal_gen(size):
    g = ((combo(n, k) for k in range(n+1)) for n in range(size+1))
    for row in g:
        for item in row:
            pass
    stats = tracemalloc.take_snapshot().statistics('lineno')
    print(stats[0].size, 'bytes')

In [ ]:
tracemalloc.stop()
tracemalloc.clear_traces()
tracemalloc.start()
pascal_list(300)

1998608 bytes
1998608 bytes


In [ ]:
tracemalloc.stop()
tracemalloc.clear_traces()
tracemalloc.start()
pascal_gen(300)

222 bytes
222 bytes


As you can see, using a generator did not require as much memory. Because we are essentially using a lazy iterator, the memory required by a previous result is released once the next iteration is requested.

### Yield From

We saw when we had two nested generators that we had to use a nested loop in order to iterate through both iterators:

In [ ]:
def matrix(n):
    gen = ( (i * j for j in range(1, n+1))
            for i in range(1, n+1)
          )
    return gen

In [ ]:
m = list(matrix(5))

In [ ]:
m

[<generator object matrix.<locals>.<genexpr>.<genexpr> at 0x0000028236EC2BF8>,
 <generator object matrix.<locals>.<genexpr>.<genexpr> at 0x0000028236EC2C50>,
 <generator object matrix.<locals>.<genexpr>.<genexpr> at 0x0000028236EC2EB8>,
 <generator object matrix.<locals>.<genexpr>.<genexpr> at 0x0000028236EC2F10>,
 <generator object matrix.<locals>.<genexpr>.<genexpr> at 0x0000028236EC29E8>]

Suppose we want an iterator to iterate over all the values of the matrix, element by element.

We could write it this way:

In [ ]:
def matrix_iterator(n):
    for row in matrix(n):
        for item in row:
            yield item

All we have done here is create a generator (iterator) that can be used to iterate of the elements of a nested iterator.

We can then use it this way:

In [ ]:
for i in matrix_iterator(3):
    print(i)

1
2
3
2
4
6
3
6
9


But we can avoid using that nested for loop by using a special form of `yield`: `yield from`

In [ ]:
def matrix_iterator(n):
    for row in matrix(n):
        yield from row

In [ ]:
for i in matrix_iterator(3):
    print(i)

1
2
3
2
4
6
3
6
9


As you can see we obtain the same result.

We can think of 
```
yield from <iterator>
```
as a replacement for the code:
```
for i in <iterator>:
    yield i
```

We'll come back to `yield from` in more detail, because there's a **lot** more to it than just a simple replacement for that inner loop!

#### Example

Here's an example where using `yield from` can be quite effective.

In this example we need to read car brands from multiple files to get it as a single collection.

We might do it this way:

In [ ]:
brands = []

with open('car-brands-1.txt') as f:
    for brand in f:
        brands.append(brand.strip('\n'))
        
with open('car-brands-2.txt') as f:
    for brand in f:
        brands.append(brand.strip('\n'))
        
with open('car-brands-3.txt') as f:
    for brand in f:
        brands.append(brand.strip('\n'))

In [ ]:
for brand in brands:
    print(brand, end=', ')

Alfa Romeo, Aston Martin, Audi, Bentley, Benz, BMW, Bugatti, Cadillac, Chevrolet, Chrysler, Citro�n, Corvette, DAF, Dacia, Daewoo, Daihatsu, Datsun, De Lorean, Dino, Dodge, Farboud, Ferrari, Fiat, Ford, Honda, Hummer, Hyundai, Jaguar, Jeep, KIA, Koenigsegg, Lada, Lamborghini, Lancia, Land Rover, Lexus, Ligier, Lincoln, Lotus, Martini, Maserati, Maybach, Mazda, McLaren, Mercedes-Benz, Mini, Mitsubishi, Nissan, Noble, Opel, Peugeot, Pontiac, Porsche, Renault, Rolls-Royce, Saab, Seat, Škoda, Smart, Spyker, Subaru, Suzuki, Toyota, Vauxhall, Volkswagen, Volvo, 

But notice that we had to load up the entire data set in memory.

As we have discussed before this is not very efficient.

Instead we could use a generator approach as follows:

In [ ]:
def brands(*files):
    for f_name in files:
        with open(f_name) as f:
            for line in f:
                yield line.strip('\n')

In [ ]:
files = 'car-brands-1.txt', 'car-brands-2.txt', 'car-brands-3.txt'
for brand in brands(*files):
    print(brand, end = ', ')

Alfa Romeo, Aston Martin, Audi, Bentley, Benz, BMW, Bugatti, Cadillac, Chevrolet, Chrysler, Citro�n, Corvette, DAF, Dacia, Daewoo, Daihatsu, Datsun, De Lorean, Dino, Dodge, Farboud, Ferrari, Fiat, Ford, Honda, Hummer, Hyundai, Jaguar, Jeep, KIA, Koenigsegg, Lada, Lamborghini, Lancia, Land Rover, Lexus, Ligier, Lincoln, Lotus, Martini, Maserati, Maybach, Mazda, McLaren, Mercedes-Benz, Mini, Mitsubishi, Nissan, Noble, Opel, Peugeot, Pontiac, Porsche, Renault, Rolls-Royce, Saab, Seat, Škoda, Smart, Spyker, Subaru, Suzuki, Toyota, Vauxhall, Volkswagen, Volvo, 

We can simplify our function by using `yield from`:

In [ ]:
def brands(*files):
    for f_name in files:
        with open(f_name) as f:
            yield from f

In [ ]:
for brand in brands(*files):
    print(brand, end=', ')

Alfa Romeo
, Aston Martin
, Audi
, Bentley
, Benz
, BMW
, Bugatti
, Cadillac
, Chevrolet
, Chrysler
, Citro�n
, Corvette
, DAF
, Dacia
, Daewoo
, Daihatsu
, Datsun
, De Lorean
, Dino
, Dodge, Farboud
, Ferrari
, Fiat
, Ford
, Honda
, Hummer
, Hyundai
, Jaguar
, Jeep
, KIA
, Koenigsegg
, Lada
, Lamborghini
, Lancia
, Land Rover
, Lexus
, Ligier
, Lincoln
, Lotus
, Martini, Maserati
, Maybach
, Mazda
, McLaren
, Mercedes-Benz
, Mini
, Mitsubishi
, Nissan
, Noble
, Opel
, Peugeot
, Pontiac
, Porsche
, Renault
, Rolls-Royce
, Saab
, Seat
, Škoda
, Smart
, Spyker
, Subaru
, Suzuki
, Toyota
, Vauxhall
, Volkswagen
, Volvo, 

Now we still have to clean up that trailing `\n` character...

So, we are going to create generators that can read each line of the file, and yield a clean result, and we'll `yield from` that generator:

In [ ]:
def gen_clean_read(file):
    with open(file) as f:
        for line in f:
            yield line.strip('\n')

As you can see, this generator function will clean each line of the file before yielding it. Let's try it with a single file and make sure it works:

In [ ]:
f1 = gen_clean_read('car-brands-1.txt')
for line in f1:
    print(line, end=', ')

Alfa Romeo, Aston Martin, Audi, Bentley, Benz, BMW, Bugatti, Cadillac, Chevrolet, Chrysler, Citro�n, Corvette, DAF, Dacia, Daewoo, Daihatsu, Datsun, De Lorean, Dino, Dodge, 

Ok, that works. So now, we can proceed with our overarching generator function as before, except we'll `yield from` our generators, instead of directly from the file iterator:

In [ ]:
files = 'car-brands-1.txt', 'car-brands-2.txt', 'car-brands-3.txt'

In [ ]:
def brands(*files):
    for file in files:
        yield from gen_clean_read(file)

In [ ]:
for brand in brands(*files):
    print(brand, end=', ')

Alfa Romeo, Aston Martin, Audi, Bentley, Benz, BMW, Bugatti, Cadillac, Chevrolet, Chrysler, Citro�n, Corvette, DAF, Dacia, Daewoo, Daihatsu, Datsun, De Lorean, Dino, Dodge, Farboud, Ferrari, Fiat, Ford, Honda, Hummer, Hyundai, Jaguar, Jeep, KIA, Koenigsegg, Lada, Lamborghini, Lancia, Land Rover, Lexus, Ligier, Lincoln, Lotus, Martini, Maserati, Maybach, Mazda, McLaren, Mercedes-Benz, Mini, Mitsubishi, Nissan, Noble, Opel, Peugeot, Pontiac, Porsche, Renault, Rolls-Royce, Saab, Seat, Škoda, Smart, Spyker, Subaru, Suzuki, Toyota, Vauxhall, Volkswagen, Volvo, 

I want to point out that in this particular instance, we are using `yield from` as a simple replacement for a `for` loop. We could equally well have written it this way:

Using `yield from`:

In [ ]:
def brands(*files):
    for file in files:
        yield from gen_clean_read(file)

Without using `yield from`:

In [ ]:
def brands(*files):
    for file in files:
        for line in gen_clean_read(file):
            yield line

In [ ]:
for brand in brands(*files):
    print(brand, end=', ')

Alfa Romeo, Aston Martin, Audi, Bentley, Benz, BMW, Bugatti, Cadillac, Chevrolet, Chrysler, Citro�n, Corvette, DAF, Dacia, Daewoo, Daihatsu, Datsun, De Lorean, Dino, Dodge, Farboud, Ferrari, Fiat, Ford, Honda, Hummer, Hyundai, Jaguar, Jeep, KIA, Koenigsegg, Lada, Lamborghini, Lancia, Land Rover, Lexus, Ligier, Lincoln, Lotus, Martini, Maserati, Maybach, Mazda, McLaren, Mercedes-Benz, Mini, Mitsubishi, Nissan, Noble, Opel, Peugeot, Pontiac, Porsche, Renault, Rolls-Royce, Saab, Seat, Škoda, Smart, Spyker, Subaru, Suzuki, Toyota, Vauxhall, Volkswagen, Volvo, 

We'll come back to `yield from` in a lot more detail later when we study coroutines - there's a whole lot more to `yield from` than a replacement for a simple loop!

### Aggregators

We have already used many built-in aggregators.

In [ ]:
def squares(n):
    for i in range(n):
        yield i**2

In [ ]:
list(squares(5))

[0, 1, 4, 9, 16]

We can find the `min` and `max` of elements in an iterable:

In [ ]:
min(squares(5))

0

In [ ]:
max(squares(5))

16

Be careful, all these aggregation functions will **exhaust** any iterator being used.

In [ ]:
sq = squares(5)

In [ ]:
max(sq)

16

In [ ]:
min(sq)

ValueError: min() arg is an empty sequence

We also have `sum`:

In [ ]:
list(squares(5))

[0, 1, 4, 9, 16]

In [ ]:
sum(squares(5))

30

#### The `any` function

The `any` function is a predicate (a function that returns `True` or `False`) that takes an iterable and returns `True` if all elements of that iterable are True (or have an associated True truth-value, i.e. **truthy**).

Remember that by default custom objects are always truthy:

In [ ]:
class Person:
    pass

In [ ]:
p = Person()

In [ ]:
bool(p)

True

For numbers, anything other than `0` is truthy, and strings, lists, tuples, dictionaries, etc are falsy if they are empty.

In fact, any empty sequence type (i.e. length = 0) is falsy, including custom sequence types:

In [ ]:
class MySeq:
    def __init__(self, n):
        self.n = n
        
    def __len__(self):
        return self.n
    
    def __getitem__(self, s):
        pass

In [ ]:
my_seq = MySeq(0)

In [ ]:
bool(my_seq)

False

In [ ]:
my_seq = MySeq(10)

In [ ]:
bool(my_seq)

True

The `any` function can be used to quickly test if any element is **truthy**:

In [ ]:
any([0, '', None])

False

In [ ]:
any([0, '', None, 'hello'])

True

Basically, the `any` function is like doing an `or` between all the elements of the iterable, and casting the result to a Boolean:

In [ ]:
result = 0 or '' or None or 'hello'
result, bool(result)

('hello', True)

#### The `all` Function

The `all` function is very similar to the `any` function, but it determines if **all** the elements of the iterable are truthy.

Basically it is equivalent to doing an `and` between all the elements oif the iterable and casting the result to a Boolean.

In [ ]:
all([1, 'abc', [1, 2], range(5)])

True

In [ ]:
all([1, 'abc', [1, 2], range(5), ''])

False

#### In Practice

In practice, we often need to test if all elements of an iterable satisfy some criteria, not necessarily whether the elements are truthy or falsy.

But we can easily apply a predicate to an iterable to first evaluate the conditions we want, and then feed that into the `any` or `all` functions.

This is where the `map` function is extremely useful! Alternatively, we can also use generator expressions.

Let's see a few examples.

##### Example 1

Suppose we want to test if an iterable contains only numeric values.

First, we need to figure out how we determine if something is a number.

This is actually a very common question on the web, with all kinds of weird and wonderful solutions - most of which actually work (for the most part).

But the simplest is to test if the object we are looking at is an instance of the `Number` class!

In [ ]:
from numbers import Number

In [ ]:
isinstance(10, Number), isinstance(10.5, Number)

(True, True)

In [ ]:
isinstance(2+3j, Number)

True

In [ ]:
from decimal import Decimal

In [ ]:
isinstance(Decimal('10.3'), Number)

True

In [ ]:
isinstance(True, Number)

True

On the other hand:

In [ ]:
isinstance('100', Number)

False

In [ ]:
isinstance([10, 20], Number)

False

Now suppose we have a list (or iterable in general) and we want to see if they are all numbers:

We could proceed with a rather clunky approach this way:

In [ ]:
l = [10, 20, 30, 40]

is_all_numbers = True
for item in l:
    if not isinstance(item, Number):
        is_all_numbers = False
        break
print(is_all_numbers)

True


In [ ]:
l = [10, 20, 30, 40, 'hello']

is_all_numbers = True
for item in l:
    if not isinstance(item, Number):
        is_all_numbers = False
        break
print(is_all_numbers)

False


Now we can actually simplify this a little, by using the `else` clause of the `for`loop - remember that the `else` clause of a `for` loop will execute if the loop terminated normally (i.e. did not `break` out of the loop).

In [ ]:
l = [10, 20, 30, 40, 'hello']
is_all_numbers = False
for item in l:
    if not isinstance(item, Number):
        break
else: # nobreak --> all numbers
    is_all_numbers = True
print(is_all_numbers)

False


Still this is clunky - there has to be a better way!

Yes, of course - the `all` function.

But we can't use it directly on the items - we're not interested in whether they are all truthy or not, we are interested in whether they are all numbers or not.

To achieve this we need to transform each element of the list using a predicate that will return `True` if the element is a number and `False` otherwise.

We can use the `map` function to apply a function (with a single parameter) to all the elements of an iterable:

In [ ]:
map(str, [0, 1, 2, 3, 4])

Now `map` is lazy, so let's put it into a list to see what it contains:

In [ ]:
list(map(str, [0, 1, 2, 3, 4]))

['0', '1', '2', '3', '4']

The function we actually want to use is the `isinstance` function - but that requires **two** parameters - the element we are testing, and the `type` we are testing for.

Somehow we need to create a form of `isinstance` that only requires a single variable and simply holds the type (`Number`) fixed.

We can do this very simply using a function or a lambda.

In [ ]:
def is_number(x):
    return is_instance(x, Number)

or, simply a lambda:

In [ ]:
lambda x: isinstance(x, Number)

<function __main__.<lambda>>

So now, let's map that function to our iterable:

In [ ]:
l

[10, 20, 30, 40, 'hello']

In [ ]:
list(map(lambda x: isinstance(x, Number), l))

[True, True, True, True, False]

And of course, **now** we can use the `all` function to determine if all the elements are numbers or not:

In [ ]:
l = [10, 20, 30, 40, 'hello']
all(map(lambda x: isinstance(x, Number), l))

False

In [ ]:
l = [10, 20, 30, 40]
all(map(lambda x: isinstance(x, Number), l))

True

A lot less typing than the first approach we did!

If you don't like using `map` for some reason, we can easily use a generator expression as well:

In [ ]:
l = [10, 20, 30, 40]
all(isinstance(x, Number) for x in l)

True

In [ ]:
l = [10, 20, 30, 40, 'hello']
all(isinstance(x, Number) for x in l)

False

Both approaches work equally well - use whichever one you are most comfortable with - but do try to use both and once you are comfortable with both approaches, then choose!

##### Example 2

Let's look at another simple example.

Suppose we have a file and we want to make sure that all the rows in the file have length > some number.

Let's just see what data we have in our sample data file:

In [ ]:
with open('car-brands.txt') as f:
    for row in f:
        print(len(row), row, end='')

11 Alfa Romeo
13 Aston Martin
5 Audi
8 Bentley
5 Benz
4 BMW
8 Bugatti
9 Cadillac
10 Chevrolet
9 Chrysler
8 Citro�n
9 Corvette
4 DAF
6 Dacia
7 Daewoo
9 Daihatsu
7 Datsun
10 De Lorean
5 Dino
5 Dodge

We can easily test to make sure that every brand in our file is at least 3 characters long:

In [ ]:
with open('car-brands.txt') as f:
    result = all(map(lambda row: len(row) >= 3, f))
print(result)

True


And we can test to see if any line is more than 10 characters:

In [ ]:
with open('car-brands.txt') as f:
    result = any(map(lambda row: len(row) > 10, f))
print(result)

True


More than 13?

In [ ]:
with open('car-brands.txt') as f:
    result = any(map(lambda row: len(row) > 13, f))
print(result)

False


Of course, we can also do this using generator expressions instead of `map`:

In [ ]:
with open('car-brands.txt') as f:
    result = any(len(row) > 13 for row in f)
print(result)

False


### Slicing Iterables

We know that sequence types can be sliced:

In [ ]:
l = [1, 2, 3, 4, 5]

In [ ]:
l[0:2]

[1, 2]

Equivalently we can use the `slice` object:

In [ ]:
s = slice(0, 2)

In [ ]:
l[s]

[1, 2]

But this does not work with iterables that are not also sequence types:

In [ ]:
import math

def factorials(n):
    for i in range(n):
        yield math.factorial(i)

In [ ]:
facts = factorials(100)

In [ ]:
facts[0:2]

TypeError: 'generator' object is not subscriptable

But we could write a function to mimic this. Let's try a simplistic approach that will only work for a consecutive slice:

In [ ]:
def slice_(iterable, start, stop):
    for _ in range(0, start):
        next(iterable)
        
    for _ in range(start, stop):
        yield(next(iterable))

In [ ]:
list(slice_(factorials(100), 1, 5))

[1, 2, 6, 24]

This is quite simple, however we don't support a `step` value.

The `itertools` module has a function, `islice` which implements this for us:

In [ ]:
list(factorials(10))

[1, 1, 2, 6, 24, 120, 720, 5040, 40320, 362880]

Now let's use the `islice` function to obtain the first 3 elements:

In [ ]:
from itertools import islice

In [ ]:
islice(factorials(10), 0, 3)

`islice` is itself a lazy iterator, so we can iterate through it:

In [ ]:
list(islice(factorials(10), 0, 3))

[1, 1, 2]

We can even use a step value:

In [ ]:
list(islice(factorials(10), 0, 10, 2))

[1, 2, 24, 720, 40320]

It does not support negative indices, or step values, but it does support None for all the arguments. The default, as expected would then be the first element, the last element, and a step of 1:

In [ ]:
list(islice(factorials(10), None, None, 2))

[1, 2, 24, 720, 40320]

This function can be very useful when dealing with infinite iterators for example.

In [ ]:
def factorials():
    index = 0
    while True:
        yield math.factorial(index)
        index += 1

Let's say we want to see the first 5 elements. We could do it the way we have up to now:

In [ ]:
facts = factorials()
for _ in range(5):
    print(next(facts))

1
1
2
6
24


Or we could use `islice` as follows:

In [ ]:
list(islice(factorials(), 5))

[1, 1, 2, 6, 24]

One thing to note is that `islice` is a lazy iterator, but when we use a `step` value, there is no magic, Python still has to call `next` on our iterable - it just doesn't always yield it back to us.

To see this, we'll add a print statement to our generator function:

In [ ]:
def factorials():
    index = 0
    while True:
        print(f'yielding factorial({index})...')
        yield math.factorial(index)
        index += 1

In [ ]:
list(islice(factorials(), 9))

yielding factorial(0)...
yielding factorial(1)...
yielding factorial(2)...
yielding factorial(3)...
yielding factorial(4)...
yielding factorial(5)...
yielding factorial(6)...
yielding factorial(7)...
yielding factorial(8)...


[1, 1, 2, 6, 24, 120, 720, 5040, 40320]

In [ ]:
list(islice(factorials(), None, 10, 2))

yielding factorial(0)...
yielding factorial(1)...
yielding factorial(2)...
yielding factorial(3)...
yielding factorial(4)...
yielding factorial(5)...
yielding factorial(6)...
yielding factorial(7)...
yielding factorial(8)...
yielding factorial(9)...


[1, 2, 24, 720, 40320]

As you can see, even though 5 elements were yielded from `islice`, it still had to call our generator 10 times!

The same thing happens if we skip elements in the slice, it still has to call next for the skipped elements:

In [ ]:
list(islice(factorials(), 5, 10))

yielding factorial(0)...
yielding factorial(1)...
yielding factorial(2)...
yielding factorial(3)...
yielding factorial(4)...
yielding factorial(5)...
yielding factorial(6)...
yielding factorial(7)...
yielding factorial(8)...
yielding factorial(9)...


[120, 720, 5040, 40320, 362880]

The other thing to watch out for is that islice is an **iterator** - which means it becomes exhausted, **even if you pass an iterable such as a list to it**!

In [ ]:
l = [1, 2, 3, 4, 5]

In [ ]:
s = islice(l, 0, 3)

In [ ]:
list(s)

[1, 2, 3]

In [ ]:
list(s)

[]

So watch out!

Furthermore, keep in mind that `islice` iterates over our iterable in order to yield the appropriate values. This means that if we use an iterator, that iterator will get consumed, and possibly exhausted:

In [ ]:
facts = factorials()

In [ ]:
next(facts), next(facts), next(facts), next(facts)

yielding factorial(0)...
yielding factorial(1)...
yielding factorial(2)...
yielding factorial(3)...


(1, 1, 2, 6)

If we now start slicing `facts` with `islice`, remember that the first four values of `facts` have already been consumed!

In [ ]:
list(islice(facts, 0, 3))

yielding factorial(4)...
yielding factorial(5)...
yielding factorial(6)...


[24, 120, 720]

And of course, `islice` further consumed our iterator:

In [ ]:
next(facts)

yielding factorial(7)...


5040

So, just something to keep in mind when we pass iterators to `islice`, and more generally to any of the functions in `itertools`.

### Selecting and Filtering Iterators

#### *filter*  and *filterfalse*

You should already be aware of the Python built-in function `filter`.

Remember that the `filter` function can work with any iterable, including of course iterators and generators.

Let's see a quick example:

In [ ]:
def gen_cubes(n):
    for i in range(n):
        print(f'yielding {i}')
        yield i**3

Now let's say we only want to use cubes that are odd.

We need a function that will return a True if the number is odd, False otherwise. (This is technically called a **predicate** by the way - any function that given an input returns True or False is called a **predicate**)

In [ ]:
def is_odd(x):
    return x % 2 == 1

Let's make sure the function works as expected:

In [ ]:
is_odd(4), is_odd(81)

(False, True)

Now we can use that function (or we could have just used a lambda as well) with the `filter` function.

Note that the `filter` function is also lazy.

In [ ]:
filtered = filter(is_odd, gen_cubes(10))

Notice that the `gen_cubes(10)` generator was not actually used (no print output).

We can however iterate through it:

In [ ]:
list(filtered)

yielding 0
yielding 1
yielding 2
yielding 3
yielding 4
yielding 5
yielding 6
yielding 7
yielding 8
yielding 9


[1, 27, 125, 343, 729]

As we can see `filtered` will drop any values where the predicate is False.

We could easily reverse this to return not-odd (i.e. even) values:

In [ ]:
def is_even(x):
    return x % 2 == 0

In [ ]:
list(filter(is_even, gen_cubes(10)))

yielding 0
yielding 1
yielding 2
yielding 3
yielding 4
yielding 5
yielding 6
yielding 7
yielding 8
yielding 9


[0, 8, 64, 216, 512]

But we had to create a new function - instead we could use the `filterfalse` function in the `itertools` module that does the same work as `filter` but retains values where the predicate is False (instead of True as the `filter` function does).

The `filterfalse` function also uses lazy evaluation.

In [ ]:
from itertools import filterfalse

In [ ]:
evens = filterfalse(is_odd, gen_cubes(10))

No print output --> lazy evaluation

In [ ]:
list(evens)

yielding 0
yielding 1
yielding 2
yielding 3
yielding 4
yielding 5
yielding 6
yielding 7
yielding 8
yielding 9


[0, 8, 64, 216, 512]


This way we can filter using the same predicate, depending on whether the result is `True` (using `filter`) or `False` (using `filterfalse`).

#### *dropwhile* and *takewhile*

The `takewhile` function in the `itertools` module will yield elements from an iterable, as long as a specific criteria (the predicate) is `True`.

As soon as the predicate is `False`, iteration is stopped - even if subsequent elements would have had a `True` predicate - this is not a filter, this basically iterate over an iterable as long as the predicate remains `True`.

As we might expect, this function also uses lazy evaluation.

In [ ]:
from math import sin, pi

def sine_wave(n):
    start = 0
    max_ = 2 * pi
    step = (max_ - start) / (n-1)
    for _ in range(n):
        yield round(sin(start), 2)
        start += step    

In [ ]:
list(sine_wave(15))

[0.0,
 0.43,
 0.78,
 0.97,
 0.97,
 0.78,
 0.43,
 0.0,
 -0.43,
 -0.78,
 -0.97,
 -0.97,
 -0.78,
 -0.43,
 -0.0]

In [ ]:
from itertools import takewhile

list(takewhile(lambda x: 0 <= x <= 0.9, sine_wave(15)))

[0.0, 0.43, 0.78]

As you can see iteration stopped at `0.78`, even though we had values later that would have had a `True` predicate. This is different from the `filter` function:

In [ ]:
list(filter(lambda x: 0 <= x <= 0.9, sine_wave(15)))

[0.0, 0.43, 0.78, 0.78, 0.43, 0.0, -0.0]

The `dropwhile` function on the other hand starts the iteration once the predicate becomes `False`:

In [ ]:
from itertools import dropwhile

In [ ]:
l = [1, 3, 5, 2, 1]

In [ ]:
list(dropwhile(lambda x: x < 5, l))

[5, 2, 1]

As you can see the iterable skipped `1` and `3` and started the iteration once the predicate was `False`. Once the iteration begins, it no longer checks the predicate, and so we ended up with `5` and `2` and `1` in the iteration.

#### The *compress* function

The compress function is essentially a filter that takes two iterables as parameters.
The first argument is the iterable (data) that will be filtered, and the second iterable contains elements (selectors), possibly of different length than the iterable being filtered. As always in Python, any object has an associated truth value, and the selectors therefore each have a truth value as well.

The resulting iterator yields elements from the data iterable where the selector at the same "position" is truthy.

A simple analogous way to look at it would be as follows using the `zip` function:


In [ ]:
data = ['a', 'b', 'c', 'd', 'e']
selectors = [True, False, 1, 0]

In [ ]:
list(zip(data, selectors))

[('a', True), ('b', False), ('c', 1), ('d', 0)]

And only retain the elements where the second value in the tuple is truthy:

In [ ]:
[item for item, truth_value in zip(data, selectors) if truth_value]

['a', 'c']

The `compress` function works the same way, except that it is evaluated lazily and returns an iterator:

In [ ]:
from itertools import compress

In [ ]:
list(compress(data, selectors))

['a', 'c']

### Infinite Iterators

There are three functions in the `itertools` module that produce infinite iterators: `count`, `cycle` and `repeat`.

In [ ]:
from itertools import (
    count,
    cycle,
    repeat, 
    islice)

#### count

The `count` function is similar to range, except it does not have a `stop` value. It has both a `start` and a `step`:

In [ ]:
g = count(10)

In [ ]:
list(islice(g, 5))

[10, 11, 12, 13, 14]

In [ ]:
g = count(10, step=2)

In [ ]:
list(islice(g, 5))

[10, 12, 14, 16, 18]

And so on. 

Unlike the `range` function, whose arguments must always be integers, `count` works with floats as well:

In [ ]:
g = count(10.5, 0.5)

In [ ]:
list(islice(g, 5))

[10.5, 11.0, 11.5, 12.0, 12.5]

In fact, we can even use other data types as well:

In [ ]:
g = count(1+1j, 1+2j)

In [ ]:
list(islice(g, 5))

[(1+1j), (2+3j), (3+5j), (4+7j), (5+9j)]

We can even use Decimal numbers:

In [ ]:
from decimal import Decimal

In [ ]:
g = count(Decimal('0.0'), Decimal('0.1'))

In [ ]:
list(islice(g, 5))

[Decimal('0.0'),
 Decimal('0.1'),
 Decimal('0.2'),
 Decimal('0.3'),
 Decimal('0.4')]

### Cycle

`cycle` is used to repeatedly loop over an iterable:

In [ ]:
g = cycle(('red', 'green', 'blue'))

In [ ]:
list(islice(g, 8))

['red', 'green', 'blue', 'red', 'green', 'blue', 'red', 'green']

One thing to note is that this works **even** if the argument is an iterator (i.e. gets exhausted after the first complete iteration over it)!

Let's see a simple example of this:

In [ ]:
def colors():
    yield 'red'
    yield 'green'
    yield 'blue'

In [ ]:
cols = colors()

In [ ]:
list(cols)

['red', 'green', 'blue']

In [ ]:
list(cols)

[]

As expected, `cols` was exhausted after the first iteration.

Now let's see how `cycle` behaves:

In [ ]:
cols = colors()
g = cycle(cols)

In [ ]:
list(islice(g, 10))

['red', 'green', 'blue', 'red', 'green', 'blue', 'red', 'green', 'blue', 'red']

As you can see, `cycle` iterated over the elements of iterator, and continued the iteration even though the first run through the iterator technically exhausted it.

##### Example

A simple application of `cycle` is dealing a deck of cards into separate hands:

In [ ]:
from collections import namedtuple

In [ ]:
Card = namedtuple('Card', 'rank suit')

In [ ]:
def card_deck():
    ranks = tuple(str(num) for num in range(2, 11)) + tuple('JQKA')
    suits = ('Spades', 'Hearts', 'Diamonds', 'Clubs')
    for suit in suits:
        for rank in ranks:
            yield Card(rank, suit)

Assume we want 4 hands, so we can think of the hands as a list containing 4 elements - each of which is itself a list containing cards.

The indices of the hands would be `0, 1, 2, 3` in the hands list:

We could certainly do it this way:

In [ ]:
hands = [list() for _ in range(4)]

In [ ]:
hands

[[], [], [], []]

In [ ]:
index = 0
for card in card_deck():
    index = index % 4
    hands[index].append(card)
    index += 1

In [ ]:
hands

[[Card(rank='2', suit='Spades'),
  Card(rank='6', suit='Spades'),
  Card(rank='10', suit='Spades'),
  Card(rank='A', suit='Spades'),
  Card(rank='5', suit='Hearts'),
  Card(rank='9', suit='Hearts'),
  Card(rank='K', suit='Hearts'),
  Card(rank='4', suit='Diamonds'),
  Card(rank='8', suit='Diamonds'),
  Card(rank='Q', suit='Diamonds'),
  Card(rank='3', suit='Clubs'),
  Card(rank='7', suit='Clubs'),
  Card(rank='J', suit='Clubs')],
 [Card(rank='3', suit='Spades'),
  Card(rank='7', suit='Spades'),
  Card(rank='J', suit='Spades'),
  Card(rank='2', suit='Hearts'),
  Card(rank='6', suit='Hearts'),
  Card(rank='10', suit='Hearts'),
  Card(rank='A', suit='Hearts'),
  Card(rank='5', suit='Diamonds'),
  Card(rank='9', suit='Diamonds'),
  Card(rank='K', suit='Diamonds'),
  Card(rank='4', suit='Clubs'),
  Card(rank='8', suit='Clubs'),
  Card(rank='Q', suit='Clubs')],
 [Card(rank='4', suit='Spades'),
  Card(rank='8', suit='Spades'),
  Card(rank='Q', suit='Spades'),
  Card(rank='3', suit='Hearts'),


You notice how we had to use the `mod` operator and an `index` to **cycle** through the hands.

So, we can use the `cycle` function instead:

In [ ]:
hands = [list() for _ in range(4)]

In [ ]:
index_cycle = cycle([0, 1, 2, 3])
for card in card_deck():
    hands[next(index_cycle)].append(card)

In [ ]:
hands

[[Card(rank='2', suit='Spades'),
  Card(rank='6', suit='Spades'),
  Card(rank='10', suit='Spades'),
  Card(rank='A', suit='Spades'),
  Card(rank='5', suit='Hearts'),
  Card(rank='9', suit='Hearts'),
  Card(rank='K', suit='Hearts'),
  Card(rank='4', suit='Diamonds'),
  Card(rank='8', suit='Diamonds'),
  Card(rank='Q', suit='Diamonds'),
  Card(rank='3', suit='Clubs'),
  Card(rank='7', suit='Clubs'),
  Card(rank='J', suit='Clubs')],
 [Card(rank='3', suit='Spades'),
  Card(rank='7', suit='Spades'),
  Card(rank='J', suit='Spades'),
  Card(rank='2', suit='Hearts'),
  Card(rank='6', suit='Hearts'),
  Card(rank='10', suit='Hearts'),
  Card(rank='A', suit='Hearts'),
  Card(rank='5', suit='Diamonds'),
  Card(rank='9', suit='Diamonds'),
  Card(rank='K', suit='Diamonds'),
  Card(rank='4', suit='Clubs'),
  Card(rank='8', suit='Clubs'),
  Card(rank='Q', suit='Clubs')],
 [Card(rank='4', suit='Spades'),
  Card(rank='8', suit='Spades'),
  Card(rank='Q', suit='Spades'),
  Card(rank='3', suit='Hearts'),


But we really can simplify this even further - why are we cycling through the indices? Why not simply cycle through the hand themselves, and append the card to the hands?

In [ ]:
hands = [list() for _ in range(4)]

In [ ]:
hands_cycle = cycle(hands)
for card in card_deck():
    next(hands_cycle).append(card)

In [ ]:
hands

[[Card(rank='2', suit='Spades'),
  Card(rank='6', suit='Spades'),
  Card(rank='10', suit='Spades'),
  Card(rank='A', suit='Spades'),
  Card(rank='5', suit='Hearts'),
  Card(rank='9', suit='Hearts'),
  Card(rank='K', suit='Hearts'),
  Card(rank='4', suit='Diamonds'),
  Card(rank='8', suit='Diamonds'),
  Card(rank='Q', suit='Diamonds'),
  Card(rank='3', suit='Clubs'),
  Card(rank='7', suit='Clubs'),
  Card(rank='J', suit='Clubs')],
 [Card(rank='3', suit='Spades'),
  Card(rank='7', suit='Spades'),
  Card(rank='J', suit='Spades'),
  Card(rank='2', suit='Hearts'),
  Card(rank='6', suit='Hearts'),
  Card(rank='10', suit='Hearts'),
  Card(rank='A', suit='Hearts'),
  Card(rank='5', suit='Diamonds'),
  Card(rank='9', suit='Diamonds'),
  Card(rank='K', suit='Diamonds'),
  Card(rank='4', suit='Clubs'),
  Card(rank='8', suit='Clubs'),
  Card(rank='Q', suit='Clubs')],
 [Card(rank='4', suit='Spades'),
  Card(rank='8', suit='Spades'),
  Card(rank='Q', suit='Spades'),
  Card(rank='3', suit='Hearts'),


#### Repeat

The `repeat` function is used to create an iterator that just returns the same value again and again. By default it is infinite, but a count can be specified optionally:

In [ ]:
g = repeat('Python')
for _ in range(5):
    print(next(g))

Python
Python
Python
Python
Python


And we also optionally specify a count to make the iterator finite:

In [ ]:
g = repeat('Python', 4)

In [ ]:
list(g)

['Python', 'Python', 'Python', 'Python']

The important thing to note as well, is that the "value" that is returned is the **same** object every time!

Let's see this:

In [ ]:
l = [1, 2, 3]

In [ ]:
result = list(repeat(l, 3))

In [ ]:
result

[[1, 2, 3], [1, 2, 3], [1, 2, 3]]

In [ ]:
l is result[0], l is result[1], l is result[2]

(True, True, True)

So be careful here. If you try to use repeat to create three separate instances of a list, you'll actually end up with shared references:

In [ ]:
result[0], result[1], result[2]

([1, 2, 3], [1, 2, 3], [1, 2, 3])

In [ ]:
result[0][0] = 100

In [ ]:
result[0], result[1], result[2]

([100, 2, 3], [100, 2, 3], [100, 2, 3])

If you want to end up with three separate copies of your argument, then you'll need to use a copy mechanism (either shallow or deep depending on your needs).

This is easily done using a comprehension expression:

In [ ]:
l = [1, 2, 3]
result = [item[:] for item in repeat(l, 3)]

In [ ]:
result

[[1, 2, 3], [1, 2, 3], [1, 2, 3]]

In [ ]:
l is result[0], l is result[1], l is result[2]

(False, False, False)

In [ ]:
result[0][0] = 100

In [ ]:
result

[[100, 2, 3], [1, 2, 3], [1, 2, 3]]

# Assignment

For this project you are given a file that contains some parking ticket violations for NYC.

(It's just a tiny extract!)

If you're wondering where I get these data sets, Kaggle is an **excellent** source of data sets in a whole variety of topics: 
https://www.kaggle.com/

You have to sign up, but it's free.

If you want the full data set, it's available here: https://www.kaggle.com/new-york-city/nyc-parking-tickets/version/2#


For this sample data set, the file is named: 
```
nyc_parking_tickets_extract.csv
```


What are your goals?

##### Goal 1
Create a lazy iterator that will return a named tuple of the data in each row. The data types should be appropriate - i.e. if the column is a date, you should be storing dates in the named tuple, if the field is an integer, then it should be stored as an integer, etc.

##### Goal 2

Calculate the number of violations by car make.

##### Note:
Try to use lazy evaluation as much as possible - it may not always be possible though! That's OK, as long as it's kept to a minimum.
